As a half-French, half-German person, I’ve often found myself discussing electricity policies in the last months, especially in the context of the climate crisis.

Germany has been heavily investing in renewable energy sources (wind and solar), which comes with its backup fossil gas sources and shutting down nuclear. 

France has been using low-carbon electricity for decades now, due to its use of nuclear energy.

There are a lot of questions I’ve been wanting to ask for a long time regarding German and French electricity. Since this morning, I’ve learnt that there is an easy way to get access to electric data using Python called [entsoe-py](https://github.com/EnergieID/entsoe-py). This notebook is a quick exploration of its capabilities. I’ll try to reproduce the analysis shown in this tweet:

<blockquote class="twitter-tweet"><p lang="fr" dir="ltr">J’ai ressorti la calculette parce que certains n’ont toujours pas compris que BotElectricity n’est pas ElectricityMap.<a href="https://t.co/1E14LVOwa5">https://t.co/1E14LVOwa5</a><br><br>Donc regardons dans le détail. On va se rapprocher des chiffres de ElectricityMap.</p>&mdash; Thomas 💉💉💉 (@Thomas_Auriel) <a href="https://twitter.com/Thomas_Auriel/status/1481741563776614406?ref_src=twsrc%5Etfw">January 13, 2022</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script> 

# Querying the production 

In [ ]:
import entsoe
import pandas as pd

api_key = open(".entsoetoken").read()

client = entsoe.EntsoePandasClient(api_key)

start = pd.Timestamp('20210601', tz='Europe/Brussels')
end = pd.Timestamp('20220113', tz='Europe/Brussels')
country_code = 'FR'  # Belgium

df = client.query_generation(country_code, start=start, end=end, psr_type=None)

df

Let’s transform the multi-index to a normal one.

In [ ]:
def convert_df(df):
    """Gets rid of the multi index"""
    cols = df.columns
    new_cols = ["_".join([col[0], col[1]]) for col in cols]
    new_df = pd.DataFrame(data=df.values, columns=new_cols, index=df.index)
    return new_df

df = convert_df(df)

And now, let’s plot the result.

In [ ]:
import hvplot.pandas

df.hvplot(responsive=True, height=500)

# Estimating CO2 impact 

As explained in the above twitter thread (in French), ENTSOE does not compute CO2 intensity per se. To do that, we need to multiply the production values by an intensity. 

We can conveniently copy the script from BotElectricity found here: https://gitlab.com/ThomasAuriel/BotElectricity/-/blob/master/scripts/data/co2Intensity.py

In [ ]:
intensities = {"Mixed": 600,
            "Generation": 600,
            "Load": 600,
            "Biomass": 230,
            "Fossil Brown coal/Lignite": 1200,  # According to https://www.energy-charts.de/emissions.htm?source=lignite&view=specific&emission=co2&year=2017 but IPCC Does no do the difference between lignite and Coal
            "Fossil Coal-derived gas": 820,
            "Fossil Gas": 490,
            "Fossil Hard coal": 820,
            "Fossil Oil": 490,
            "Fossil Oil shale": 490,
            "Fossil Peat": 820,
            "Geothermal": 38,
            "Hydro Pumped Storage": 24,
            "Hydro Run-of-river and poundage": 24,
            "Hydro Water Reservoir": 24,
            "Marine": 24,
            "Nuclear": 12,
            "Other renewable": 30,
            "Solar": 48,
            "Waste": 230,
            "Wind Offshore": 12,
            "Wind Onshore": 11,
            "Other": 600,
}

And now, let’s map this and sum it to get an average CO2 intensity in gCO2/kWh.

In [ ]:
import numpy as np

def compute_intensity(df):
    co2_intensity = pd.Series(data=df.iloc[:, 0] * 0, index=df.index, name="CO2 intensity gCO2/kWh")
    for col in df.columns:
        label = col.split("_")[0]
        intensity = intensities[label]
        co2_intensity += intensity * df[col].replace(np.nan, 0)
    co2_intensity /= df.iloc[:, :-1].sum(axis=1).values
    return co2_intensity

co2_intensity = compute_intensity(df)

co2_intensity.hvplot(responsive=True, height=500)

# Case study France vs Germany 

In [ ]:
start = pd.Timestamp('20181201', tz='Europe/Brussels')
end = pd.Timestamp('20220101', tz='Europe/Brussels')

data = {}
for country_code in ["FR", "DE"]: 
    df = client.query_generation(country_code, start=start, end=end, psr_type=None)
    df = convert_df(df)
    data[country_code] = compute_intensity(df)

In [ ]:
def resample(df, rule="W"):
    c = pd.concat([df.resample(rule="W").min(),
           df.resample(rule="W").mean(), 
           df.resample(rule="W").max(),],
          axis=1)
    c.columns = ["min", "mean", "max"]
    return c 

resample(data["FR"]).hvplot(label="FR",  responsive=True, height=500) * resample(data["DE"]).hvplot(label="DE", ylabel="CO2 intensity gCO2/kWh", responsive=True, height=500)  

Compared to Germany, France shines with really low CO2 intensity. Of course, one has to ask, at what price? Is the nuclear risk worth taking? 

On the other side, seing this rather CO2 high intensity in Germany, one could ask: so 1000 Billion $ got you this kind of "green electricity"? Where did the money go because it doesn’t look green to me, although a lot of it is renewable. But that means the German electricity is contributing more to Climate Change (4x more).

# Subtleties

- production vs consumption: here come exports
- prices: looking at electricity map, it seems prices in France are way higher than elsewhere -- what are these prices? who pays them? what do they mean?